<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Multi-label-Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm==0.4.5 -q

     |████████████████████████████████| 296kB 6.3MB/s 


In [2]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Train.7z
!7z x /content/Train.7z

--2021-04-16 14:53:02--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Train.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16048162 (15M) [application/octet-stream]
Saving to: ‘Train.7z’

Train.7z            100%[===================>]  15.30M  47.9MB/s    in 0.3s    

2021-04-16 14:53:03 (47.9 MB/s) - ‘Train.7z’ saved [16048162/16048162]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 16048162 bytes (16 MiB)

Extracting archive: /content/Train.7z
--
Path = /content/Train.7z
Ty

In [3]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Val.7z
!7z x /content/Val.7z

--2021-04-16 14:53:05--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Val.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5118157 (4.9M) [application/octet-stream]
Saving to: ‘Val.7z’

Val.7z              100%[===================>]   4.88M  29.5MB/s    in 0.2s    

2021-04-16 14:53:05 (29.5 MB/s) - ‘Val.7z’ saved [5118157/5118157]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 5118157 bytes (4999 KiB)

Extracting archive: /content/Val.7z
--
Path = /content/Val.7z
Type = 7z
Phy

In [4]:
import timm
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.preprocessing import LabelEncoder
import os,time
from math import fsum
from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image
from torchvision import transforms

## Cleaning and getting a final data

In [ ]:
from PIL import Image
from glob import glob
images = [x for x in glob('./val/*/*') if 'val.csv' not in x]
for i in images:
  Image.open(i).convert('RGB').save(i)

In [15]:
labels = ["phylum","class","species","form","sample"]

train = pd.read_csv('/content/train/train.csv')
val = pd.read_csv('/content/val/val.csv')

train = train[train['species']!='na']
val = val[val['species']!='na']

In [16]:
label_encoders = dict()
for x in labels:
  le = LabelEncoder()
  train[f'{x}_label'] = le.fit_transform(train[x])
  val[f'{x}_label'] = le.transform(val[x])
  label_encoders[x] = le

In [17]:
train.to_csv('/content/train.csv',index = False)

val.to_csv('/content/val.csv',index = False)

# Dataset Objects

In [8]:
transformer = transforms.Compose([
        transforms.Resize((224,224)),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

In [9]:
class MultiDataset(Dataset):
  def __init__(self,dataFrame,label_encoders,transform=None):
    self.data = dataFrame
    self.transform = transform
    self.labels = ["phylum_label","class_label","species_label","form_label","sample_label"]
    self.label_encoder = label_encoders
    self.nclasses = self.get_nclasses()
    total = fsum(self.nclasses.values())

    self.weights = {k:v/total for k,v in self.nclasses.items()}

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img_path = self.data.loc[idx,"img_path"]
    image = Image.open(img_path)
    image = self.transform(image)
    labels = self.data.loc[idx,self.labels].to_dict()
    for label in labels:
      labels[label] = torch.tensor(labels[label],dtype = torch.long)
            
    dict_data = {
    'image': image,
    'labels': labels
    }
    return dict_data
  
  # def get_label_encode(self):
  #   d=dict()
  #   for label in self.labels:            
  #       le = LabelEncoder()
  #       self.data[label]=le.fit_transform(self.data[label])
  #       d[label]=le
  #   return d
    
  def get_nclasses(self):
    return self.data[self.labels].nunique().to_dict()

# Multi-label Classification

In [28]:
class MultiViT(nn.Module):
  def __init__(self,nclasses,labels):
    super(MultiViT,self).__init__()


    self.img_transformer = timm.models.vision_transformer.vit_base_patch16_224_in21k(pretrained=True,num_classes = 0)
    # self.img_transformer.load_state_dict(torch.load(vit_path))
    self.fc1 = nn.Linear(768,3*768)
    self.mha = nn.MultiheadAttention(embed_dim = 768,
                                     num_heads = 6,
                                     dropout = 0.38,
                                     bias = False)
    # for param in self.img_transformer.parameters():
    #   param.requires_grad = False
    # self.drop = nn.Dropout(0.38)
    self.classifiers = nn.ModuleDict()
    for label in labels:
      self.classifiers[label]=nn.Sequential(nn.Linear(768,512),
                                            nn.ReLU(),
                                            nn.Linear(512,nclasses[label]))

    
  def forward(self,x):
    x=self.img_transformer(x)
    x = self.fc1(x)
    x = F.relu(x)
    x = x.unsqueeze(dim = 0)
    q, k, v = x.split(768,dim = 2)
    z,_ = self.mha(q,k,v)
    z = z.squeeze(dim = 0)
    

    # x=self.drop(x)
    y=dict()
    for label in self.classifiers.keys():
        y[label]=self.classifiers[label](z)
    # print(y.keys())
    return y


In [18]:
train = pd.read_csv("/content/train.csv")
val = pd.read_csv("/content/val.csv")

train_dataset = MultiDataset(train,label_encoders,transformer)
val_dataset = MultiDataset(val,label_encoders,transformer)

train_loader = DataLoader(train_dataset,batch_size=24)
val_loader = DataLoader(val_dataset,batch_size=17)

In [29]:
device = torch.device('cuda')
multimodel = MultiViT(train_dataset.nclasses,train_dataset.labels)
multimodel.to(device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, multimodel.parameters()),lr = 1e-4)

Removing representation layer for fine-tuning.


In [13]:
#@title Training Functions
def get_loss(out,labels,loss_fn,weights):
    loss=0
    for label in labels:
        loss+=loss_fn(out[label],labels[label])*weights[label]
    return loss


def cal_accuracy(out,labels,batch_size,epoch_acc):
    for key in out:
        q=(out[key].detach().argmax(axis=1)==labels[key]).sum().item()/batch_size
        if not key in epoch_acc.keys():
            epoch_acc[key]=q
        else:
            epoch_acc[key]+=q
    return epoch_acc


def get_avg_acc(epoch_acc,loader):
    acc=0
    for key in epoch_acc:
        epoch_acc[key]/=len(loader)
        acc+=epoch_acc[key]
        
    return acc/len(epoch_acc),epoch_acc

def evaluate(model,loss_fn,loader):
    model.eval()
    epoch_acc={}
    epoch_loss=0
    with torch.no_grad():
      for batch in loader:
          img=batch["image"].to(device)
          labels=batch["labels"]
          for key in labels:
              labels[key]=labels[key].to(device)
          
          out=model(img)
          loss=get_loss(out,labels,loss_fn,loader.dataset.weights)
          epoch_loss+=loss.item()
          epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
        
    avg_acc,epoch_acc=get_avg_acc(epoch_acc,loader)
    return (epoch_loss/len(loader),avg_acc,epoch_acc)

def train_model(model, loss_fn, opt,trainloader,valloader, num_epochs=1):
    stop_value = 10
    since = time.time()
    best_model=""
    max_loss=np.inf
    train={"loss":[],"avg":[],"individual":[]}
    val={"loss":[],"avg":[],"individual":[]}
    for epoch in range(num_epochs):
        print("-"*50)
        print('Epoch {} of {}'.format(epoch+1, num_epochs))
        epoch_loss=0
        epoch_acc={}
        model.train()
        # Iterate over data.
        for i,data in enumerate(trainloader):
            opt.zero_grad()
            img=data["image"].to(device)
            labels=data["labels"]
            for key in labels:
                labels[key]=labels[key].to(device)
            

            out=model(img)
            # print(out,labels,sep = '\n\n')
            loss=get_loss(out,labels,loss_fn,trainloader.dataset.weights)
            epoch_loss+=loss.detach().item()
            loss.backward()
            opt.step()
            epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
            
        val_result=evaluate(model,loss_fn,valloader)
        avg_acc,epoch_acc=get_avg_acc(epoch_acc,trainloader)
        avg_loss=epoch_loss/len(trainloader)
        print('Train')
        print(f"Loss: {avg_loss:.3f}\nAccuracy:\n\tEach_label_acc->{epoch_acc}\n\tAvg acc->{avg_acc:.3f}\n")
        print("-"*200)
        print("Validation")
        print(f"Loss: {val_result[0]:.3f}\nAccuracy: Each_label_acc->{val_result[2]}\n\tAvg acc->{val_result[1]:.3f}")
        
        if val_result[0]<max_loss:
          train["avg"].append(avg_acc)
          train["individual"].append(epoch_acc)
          train["loss"].append(avg_loss)
        
          val["avg"].append(val_result[1])
          val["individual"].append(val_result[2])
          val["loss"].append(val_result[0])
        
          max_loss=val_result[0]
          torch.save(model.state_dict(),'multimodal.bin')
          print('Saving weights')
          stop_value = 10
        if val_result[0]>max_loss:
          stop_value -=1
          print(f'Patience - {stop_value}')

        if stop_value<=0:
          print("\nStopping")
          return train,val,best_model,max_loss 
        print("-"*100)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return train,val,best_model,max_loss

In [31]:
train,val,best_model,best_acc=train_model(multimodel, loss_fn, optim,train_loader,val_loader,num_epochs=140)

--------------------------------------------------
Epoch 1 of 140
Train
Loss: 0.036
Accuracy:
	Each_label_acc->{'phylum_label': 0.9952380952380951, 'class_label': 0.9952380952380951, 'species_label': 0.9958333333333332, 'form_label': 0.9946428571428572, 'sample_label': 0.994047619047619}
	Avg acc->0.995

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Validation
Loss: 0.932
Accuracy: Each_label_acc->{'phylum_label': 0.878787878787879, 'class_label': 0.8360071301247773, 'species_label': 0.704748014908443, 'form_label': 0.7254901960784316, 'sample_label': 0.8698752228163994}
	Avg acc->0.803
Saving weights
----------------------------------------------------------------------------------------------------
--------------------------------------------------
Epoch 2 of 140
Train
Loss: 0.032
Accuracy:
	Each_label_acc->{'phylum_label': 0.99821

In [32]:
multimodel = MultiViT(train_dataset.nclasses,train_dataset.labels)
multimodel.load_state_dict(torch.load('/content/multimodal.bin'))
multimodel.to(device)
preds, true = [],[]
val_loader = DataLoader(val_dataset,batch_size=17)
multimodel.eval()
with torch.no_grad():
  for batch in val_loader:
    img, labels = batch['image'], batch['labels']
    img = img.to(device)
    pred = multimodel(img)
    preds.append(pred)
    true.append(labels)


Removing representation layer for fine-tuning.


In [25]:
y_pred = {'phylum_label':[], 'class_label':[], 'species_label':[], 'form_label':[], 'sample_label':[]}
y_true = {'phylum_label':[], 'class_label':[], 'species_label':[], 'form_label':[], 'sample_label':[]}

In [26]:
for i in range(len(preds)):
  for key in y_pred:
    y_pred[key].extend(preds[i][key].argmax(axis = 1).cpu().tolist())
    y_true[key].extend(true[i][key].tolist())

In [ ]:
np.unique(y_pred[label])

array([0, 1, 2, 3, 4])

In [ ]:
label_encoders['form'].inverse_transform(np.unique(y_pred['form_label']))

array(['adult', 'cyst', 'egg', 'gametocyte', 'larva', 'oocysts',
       'sarcocysts', 'schizont', 'trophozoites'], dtype=object)

In [27]:
from sklearn.metrics import classification_report, confusion_matrix
y = '_label'
for label in y_pred:
  label_le = label.replace(y,'')
  classes = label_encoders[label_le].inverse_transform(np.unique(y_true[label]))
  print(f"\nClassification Report for {label}:\n")
  # try:
  print(classification_report(y_true[label],y_pred[label],target_names=classes))
  # except Exception as e:
  #   print(classification_report(y_true[label],y_pred[label]))
  print('='*95)


Classification Report for phylum_label:

                   precision    recall  f1-score   support

        Amoebozoa       0.83      0.83      0.83        35
      Apicomplexa       0.96      0.98      0.97       255
         Nematoda       0.85      0.81      0.83       150
  Platyhelminthes       0.81      0.86      0.84        80
Sarcomastigophora       0.93      0.80      0.86        35

         accuracy                           0.90       555
        macro avg       0.88      0.86      0.87       555
     weighted avg       0.90      0.90      0.90       555


Classification Report for class_label:

               precision    recall  f1-score   support

 Aconoidasida       1.00      0.98      0.99       180
      Cestoda       0.86      0.78      0.82        65
  Chromadorea       0.72      0.78      0.75       125
  Conoidasida       0.84      0.97      0.90        75
      Enoplea       0.65      0.44      0.52        25
Rhabditophora       0.88      1.00      0.94        

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
